# Import Modules

In [1]:
import os
import glob

os.chdir("..")

# Build Ground Truths

In [2]:
from utils.dataset import write_ground_truth

In [3]:
dataset_files = glob.glob("./datasets/potrika/*.csv")

In [4]:
write_ground_truth(dataset_files, "./datasets/potrika_gt.txt")

  0%|          | 0/8 [00:00<?, ?it/s]

2024-01-30 18:02:42.060 | INFO     | utils.dataset:write_ground_truth:46 - Total lines: 691331


# Build Incorrects

In [5]:
text_lines = None
with open("./datasets/potrika_gt.txt") as file:
    text_lines = file.readlines()
    text_lines = [text.rstrip() for text in text_lines]

In [6]:
from utils.text import apply_errors_on_many_sentences
from utils.dataset import write_incorrects

In [7]:
text_lines[:10]

['কিন্তু ফের অসুস্থ বোধ করায় তাকে আবারও হাসপাতালে ভর্তি করা হয়।',
 'তাঁর মতেও, কেন সবসময় যৌনতার প্রতীক হিসেবে মেয়েদেরকেই দেখানো হবে?',
 '‘মনের মতো মানুষ পাইলাম না’ চলচ্চিত্রের জন্য যৌথভাবে শ্রেষ্ঠ শিল্প নির্দেশক হয়েছেন রহমত উল্লাহ বাসু ও ফরিদ আহমেদ।',
 'শাশ্বত সচদেব এবং জাসলিন রয়াল এর সঙ্গীত পরিচালনা করেছেন।',
 'সাজিদ খানের বিরুদ্ধে বলিউড অভিনেত্রী র\u200c্যাচেল হোয়াইটের যৌন হয়রানির অভিযোগ শুনে ক্ষেপে যান অক্ষয় কুমার।',
 'ফলে কোন চ্যানেলে কোন নাটক প্রচারিত হচ্ছে তাও বুঝতে পারছে না দর্শক।',
 'এই গানটিরও সঙ্গীত আয়োজন করেছেন রাজা ক্যাশেফ।',
 'আশীর্বাদ সিনেমায় মাহির বিপরীতে চুক্তিবদ্ধ হয়েছেন রোশান।',
 'অনেক জনপ্রিয় শিল্পী এতে অভিনয় করছেন।',
 'তার কণ্ঠে অসংখ্য নজরুল সঙ্গীত শ্রোতারা শুনেছেন।']

In [8]:
modified_lines = apply_errors_on_many_sentences(text_lines)

  0%|          | 0/691331 [00:00<?, ?it/s]

2024-01-30 18:03:02.395 | INFO     | utils.text:apply_errors_on_many_sentences:179 - No change count: 102141


In [9]:
modified_lines[:10]

['কিন্তু ফের অসুস্থ বোধ করায় তাকে আবারও হাসপাতালে  ভর্তি  করা  হয় ।',
 'তাঁর ড়মতেও,ভ কেন সবসময় যৌনতার প্রতীক হিসেউবঠে মেয়েদেরকযেই রদকেখাসনো হবে?',
 '‘মনের মতোভ মানুষ প়াএইলাম না’ চলচ্চিত্রের জন্য যৌথভামবে শ্রেষ্ঢ়ঠ শিল্প নির্দেশক হয়েছেন রহমত উঞল্লাহ বাসুঠ ও ফযরিদ আহমেদ।',
 'শাশশ্বত সচদেব এবং  জাসললিন রয়াল এর সসঙ্গগীত  পরিচালনা কেরছেন।',
 'সাজিদ খ ানভের বিরুদ্ধে বলউিউডর অখ ভিনমেড়ত্রী রন\u200c্যাচওেল হোয়াইটের যৌন হয়রানি র অভিযোগ শইুনে ক্ষেপে যান অক্ষয় কু ম ার।',
 'লফে কোন চ্যানেলে কোন নাঠক প্রচারিত হচ্ছে তাও বুঝতে পারছে না দর্শক।',
 'এই  গ াটির ও সঙ্গ ী থ আয়োজন করেছেন াজা ক্যশেফ।',
 'আশীর্বাধ সিনেমায় মাহির বিপরীতে চুক্তিবদ্ধ হয়েছেন রোশান।',
 'অনেক জনপ্রিয় শিল্পী এতে অভিনয় করছেন।',
 'তার কণ্ঠে  অ সংখ্ য নজরুল সঙ ্ গী ত শ ্রোতারা শুনে ছেন।']

In [10]:
write_incorrects(modified_lines, "./datasets/potrika_incorrect.txt")

2024-01-30 18:03:02.999 | INFO     | utils.dataset:write_incorrects:53 - Total lines: 691331


In [11]:
exit()

# Generate Huggingface Dataset

### Dump Pairs

In [1]:
import os

os.chdir("..")

In [2]:
import json

In [3]:
def load_txt_files(path):
    with open(path) as file:
        lines = file.readlines()
        lines = [line.rstrip() for line in lines]
    return lines

In [4]:
def split_dataset_into(dataset: list, proportion: float = 0.8):
    train_size = int(len(dataset) * proportion)
    return dataset[:train_size], dataset[train_size:]

In [5]:
def write_to_jsonl(dataset, path: str):
    with open(path, "w") as file:
        json.dump(dataset, file)
    print("Done!")

In [6]:
to_sentences = load_txt_files("./datasets/potrika_gt.txt")
from_sentences = load_txt_files("./datasets/potrika_incorrect.txt")
dataset = [{"from": k, "to": v} for k, v in zip(from_sentences, to_sentences)]

train_dataset, val_dataset = split_dataset_into(dataset, 0.8)
val_dataset, test_dataset = split_dataset_into(val_dataset, 0.5)

In [7]:
write_to_jsonl(train_dataset, "./datasets/correction_train.jsonl")
write_to_jsonl(test_dataset, "./datasets/correction_test.jsonl")
write_to_jsonl(val_dataset, "./datasets/correction_val.jsonl")

Done!
Done!
Done!


In [8]:
exit()

## Generate HF Dataset

In [1]:
import os

os.chdir("..")

In [2]:
from datasets import load_dataset

In [3]:
split_config = {
    "train": "./datasets/correction_train.jsonl",
    "test": "./datasets/correction_test.jsonl",
    "val": "./datasets/correction_val.jsonl",
}

In [4]:
dataset = load_dataset("json", data_files=split_config)

In [8]:
dataset = dataset.map(
    lambda x: {"from": "00xx00" + x["from"], "to": "00xx00" + x["to"]}
)

Map:   0%|          | 0/553064 [00:00<?, ? examples/s]

Map:   0%|          | 0/69134 [00:00<?, ? examples/s]

Map:   0%|          | 0/69133 [00:00<?, ? examples/s]

In [9]:
dataset["train"][0]

{'from': '00xx00কিন্তু ফের অসুস্থ বোধ করায় তাকে আবারও হাসপাতালে  ভর্তি  করা  হয় ।',
 'to': '00xx00কিন্তু ফের অসুস্থ বোধ করায় তাকে আবারও হাসপাতালে ভর্তি করা হয়।'}